In [1]:
# import the Profile class
from voting.profiles import Profile
from voting.generate_profiles import *
from voting.voting_methods import *
import networkx as nx
from itertools import combinations
import matplotlib.pyplot as plt

In [2]:
pm="IC"
num_trials = 1000
num_candidates = 5
num_voters = 1001

num_condorcet_winner = 0
num_cycles = 0
for t in range(num_trials): 
    prof = generate_profile(num_candidates, num_voters, probmod = pm)
    num_condorcet_winner += prof.condorcet_winner() is not None
    num_cycles += has_cycle(prof.margin_graph())
    
print(f"{round(num_condorcet_winner / num_trials, 2) *100}% of profiles have a Condorcet winner.")
print(f"{round(num_cycles / num_trials, 2) *100}% of profiles have a majority cycle.")

74.0% of profiles have a Condorcet winner.
48.0% of profiles have a majority cycle.


In [3]:
pm="IAC"
num_trials = 1000
num_candidates = 5
num_voters = 1001

num_condorcet_winner = 0
num_cycles = 0
for t in range(num_trials): 
    prof = generate_profile(num_candidates, num_voters, probmod = pm)
    num_condorcet_winner += prof.condorcet_winner() is not None
    num_cycles += has_cycle(prof.margin_graph())
print(f"{round(num_condorcet_winner / num_trials, 2) *100}% of profiles have a Condorcet winner.")
print(f"{round(num_cycles / num_trials, 2) * 100}% of profiles have a majority cycle.")

76.0% of profiles have a Condorcet winner.
44.0% of profiles have a majority cycle.


In [4]:
%%time 

all_num_candidates = [3, 4, 5, 6]
all_num_voters = [(10,11), (100,101), (1000, 1001)]
num_trials = 10000

voting_scenarios = list(product(all_num_candidates, all_num_voters))
data = {vs: 
        {
            "perc_condorcet_winners": {"IC": 0, "IAC": 0},
            "perc_cycles": {"IC": 0, "IAC": 0},            
        } 
        for vs in voting_scenarios
       }

for vs in voting_scenarios: 
    nc = vs[0]
    nv_even = vs[1][0]
    nv_odd = vs[1][1]
    
    num_condorcet_winner_IC = 0
    num_condorcet_winner_IAC = 0
    num_cycle_IC = 0
    num_cycle_IAC = 0
    for t in range(num_trials): 
        prof_IC_even = generate_profile(nc, nv_even, probmod="IC")       
        prof_IC_odd = generate_profile(nc, nv_odd, probmod="IC")
        prof_IAC_even = generate_profile(nc, nv_even, probmod="IAC")        
        prof_IAC_odd = generate_profile(nc, nv_odd, probmod="IAC")
        
        num_condorcet_winner_IC += prof_IC_even.condorcet_winner() is not None 
        num_condorcet_winner_IC += prof_IC_odd.condorcet_winner() is not None 
        num_cycle_IC += has_cycle(prof_IC_even)
        num_cycle_IC += has_cycle(prof_IC_odd) 
        
        num_condorcet_winner_IAC += prof_IAC_even.condorcet_winner() is not None 
        num_condorcet_winner_IAC += prof_IAC_odd.condorcet_winner() is not None 
        num_cycle_IAC += has_cycle(prof_IAC_even)
        num_cycle_IAC += has_cycle(prof_IAC_odd) 
        
    data[vs]["perc_condorcet_winners"]["IC"] = num_condorcet_winner_IC / (2 * num_trials)
    data[vs]["perc_cycles"]["IC"] = num_cycle_IC /  (2 * num_trials)
    data[vs]["perc_condorcet_winners"]["IAC"] = num_condorcet_winner_IAC /  (2 * num_trials)
    data[vs]["perc_cycles"]["IAC"] = num_cycle_IAC /  (2 * num_trials)


CPU times: user 27min 30s, sys: 4 s, total: 27min 34s
Wall time: 27min 39s


In [5]:
import tabulate
table = list()
headers = list()

for nc in all_num_candidates: 
    row = [nc]
    for nv in all_num_voters: 
        row.append(f"{data[(nc, nv)]['perc_condorcet_winners']['IC']}, {data[(nc, nv)]['perc_condorcet_winners']['IAC']}")
    table.append(row)
headers = [str(nv) for nv in all_num_voters]
print(tabulate.tabulate(table, headers, tablefmt="github"))
                   

|    | (10, 11)        | (100, 101)      | (1000, 1001)     |
|----|-----------------|-----------------|------------------|
|  3 | 0.7547, 0.8389  | 0.8545, 0.9221  | 0.89105, 0.93385 |
|  4 | 0.65785, 0.6914 | 0.7621, 0.81035 | 0.8096, 0.8324   |
|  5 | 0.5876, 0.58815 | 0.68315, 0.701  | 0.7311, 0.7444   |
|  6 | 0.52625, 0.5295 | 0.6204, 0.622   | 0.6683, 0.6696   |


In [6]:
print("Generated by the URN model with alpha = 0")
prof = generate_profile(4, 5, probmod="URN", probmod_param=0)
prof.display()

print("Generated by the URN model with alpha = 1")
prof = generate_profile(4, 5, probmod="URN", probmod_param=1)
prof.display()

print("Generate by URN model with alpha = 10")
prof = generate_profile(4, 5, probmod="URN", probmod_param=10)
prof.display()

print("Generate by URN model with alpha = 100")
prof = generate_profile(4, 5, probmod="URN", probmod_param=100)
prof.display()

Generated by the URN model with alpha = 0
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 3 | 0 | 3 | 0 | 3 |
| 1 | 2 | 0 | 3 | 1 |
| 2 | 3 | 2 | 2 | 0 |
| 0 | 1 | 1 | 1 | 2 |
+---+---+---+---+---+
Generated by the URN model with alpha = 1
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 0 | 2 | 1 | 3 | 2 |
| 1 | 1 | 2 | 1 | 3 |
| 3 | 0 | 0 | 2 | 0 |
| 2 | 3 | 3 | 0 | 1 |
+---+---+---+---+---+
Generate by URN model with alpha = 10
+---+---+---+
| 3 | 1 | 1 |
+---+---+---+
| 0 | 3 | 0 |
| 2 | 0 | 1 |
| 1 | 1 | 2 |
| 3 | 2 | 3 |
+---+---+---+
Generate by URN model with alpha = 100
+---+
| 5 |
+---+
| 3 |
| 2 |
| 0 |
| 1 |
+---+


In [7]:
## from preflib tools ###

# Generate votes based on the URN Model..
# we need numvotes with replace replacements.
def gen_urn(numvotes, replace, alts):
    voteMap = {}
    ReplaceVotes = {}
    
    ICsize = math.factorial(len(alts))
    print("ICsize ", ICsize)
    ReplaceSize = 0

    for x in range(numvotes):
        print("start voteMap", voteMap)
        print("start ReplaceVotes", ReplaceVotes)
        flip = random.randint(1, ICsize+ReplaceSize)
        print("flip", flip)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            tvote = gen_ic_vote(alts)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
            print("ReplaceSize", ReplaceSize)
            print("made " + str(tvote))
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            print("checking replacement: flip is  ", flip)
            for vote in ReplaceVotes.keys():
                print("testing ", vote)
                flip = flip - ReplaceVotes[vote]
                print("flip is now", flip)
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")		
                exit()
        print("end voteMap", voteMap)
        print("end ReplaceVotes", ReplaceVotes)
        print("======\n")
    return voteMap

# Return a TUPLE! IC vote given a vector of alternatives.  Basically remove one
def gen_ic_vote(alts):
    options = list(alts)
    vote = []
    while(len(options) > 0):
        #randomly select an option
        vote.append(options.pop(random.randint(0,len(options)-1)))
    return tuple(vote)


In [8]:
alts = [0,1,2,3]
print(gen_ic_vote([0,1,2,3]))
print(gen_urn(3,5,alts))

(1, 3, 2, 0)
ICsize  24
start voteMap {}
start ReplaceVotes {}
flip 13
ReplaceSize 5
made (1, 2, 3, 0)
end voteMap {(1, 2, 3, 0): 1}
end ReplaceVotes {(1, 2, 3, 0): 5}

start voteMap {(1, 2, 3, 0): 1}
start ReplaceVotes {(1, 2, 3, 0): 5}
flip 2
ReplaceSize 10
made (0, 3, 1, 2)
end voteMap {(1, 2, 3, 0): 1, (0, 3, 1, 2): 1}
end ReplaceVotes {(1, 2, 3, 0): 5, (0, 3, 1, 2): 5}

start voteMap {(1, 2, 3, 0): 1, (0, 3, 1, 2): 1}
start ReplaceVotes {(1, 2, 3, 0): 5, (0, 3, 1, 2): 5}
flip 31
checking replacement: flip is   7
testing  (1, 2, 3, 0)
flip is now 2
testing  (0, 3, 1, 2)
flip is now -3
end voteMap {(1, 2, 3, 0): 1, (0, 3, 1, 2): 2}
end ReplaceVotes {(1, 2, 3, 0): 5, (0, 3, 1, 2): 10}

{(1, 2, 3, 0): 1, (0, 3, 1, 2): 2}


In [9]:
print("Generated by the MALLOWS model with alpha = 0")
prof = generate_profile(4, 5, probmod="MALLOWS", probmod_param=0)
prof.display()

print("Generated by the MALLOWS model with alpha = 0.1")
prof = generate_profile(4, 5, probmod="MALLOWS", probmod_param=0.1)
prof.display()

print("Generated by the MALLOWS model with alpha = 0.5")
prof = generate_profile(4, 5, probmod="MALLOWS", probmod_param=0.5)
prof.display()

print("Generated by the MALLOWS model with alpha = 0.8")
prof = generate_profile(4, 5, probmod="MALLOWS", probmod_param=0.8)
prof.display()


print("Generated by the MALLOWS model with alpha = 1")
prof = generate_profile(4, 5, probmod="MALLOWS", probmod_param=1.0)
prof.display()


Generated by the MALLOWS model with alpha = 0
+---+
| 5 |
+---+
| 3 |
| 1 |
| 0 |
| 2 |
+---+
Generated by the MALLOWS model with alpha = 0.1
+---+---+---+
| 1 | 2 | 2 |
+---+---+---+
| 3 | 3 | 3 |
| 2 | 0 | 0 |
| 0 | 2 | 1 |
| 1 | 1 | 2 |
+---+---+---+
Generated by the MALLOWS model with alpha = 0.5
+---+---+---+
| 3 | 1 | 1 |
+---+---+---+
| 0 | 0 | 0 |
| 2 | 1 | 3 |
| 1 | 2 | 2 |
| 3 | 3 | 1 |
+---+---+---+
Generated by the MALLOWS model with alpha = 0.8
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 0 | 0 | 3 | 0 | 3 |
| 3 | 2 | 1 | 1 | 0 |
| 2 | 1 | 2 | 3 | 1 |
| 1 | 3 | 0 | 2 | 2 |
+---+---+---+---+---+
Generated by the MALLOWS model with alpha = 1
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 3 | 0 | 2 | 0 | 2 |
| 0 | 2 | 0 | 3 | 0 |
| 1 | 1 | 3 | 1 | 1 |
| 2 | 3 | 1 | 2 | 3 |
+---+---+---+---+---+


In [10]:
def voter_utility(v_pos, c_pos, beta):
    '''Based on the Rabinowitz and Macdonald (1989) mixed model
    described in Section 3, pp. 745 - 747 of 
    "Voting behavior under the directional spatial model of electoral competition" by S. Merrill III 
    
    beta = 1 is the proximity model
    beta = 0 is the directional model
    '''
    return 2 * np.dot(v_pos, c_pos) - beta*(np.linalg.norm(v_pos)**2 + np.linalg.norm(c_pos)**2)

def create_prof_spatial_model(num_voters, cmap, params):
    num_dim = params[0] # the first component of the parameter is the number of dimensions
    beta = params[1] # used to define the mixed model: beta = 1 is proximity model (i.e., Euclidean distance)
    num_cands = len(cmap.keys())  
    mean = [0] * num_dim # mean is 0 for each dimension
    cov = np.diag([1]*num_dim)  # diagonal covariance
    
    # sample candidate/voter positions using a multivariate normal distribution
    cand_positions = np.random.multivariate_normal(np.array(mean), cov, num_cands)
    voter_positions = np.random.multivariate_normal(np.array(mean), cov, num_voters)
    
    # generate the rankings and counts for each ranking
    ranking_counts = dict()
    for v,v_pos in enumerate(voter_positions):
        v_utils = {voter_utility(v_pos,c_pos,beta): c for c,c_pos in enumerate(cand_positions)}
        ranking = tuple([v_utils[_u] for _u in sorted(v_utils.keys(),reverse=True)])
        if ranking in ranking_counts.keys():
            ranking_counts[ranking] += 1
        else:
            ranking_counts.update({ranking:1})
    
    # list of tuples where first component is a ranking and the second is the count
    prof_counts = ranking_counts.items()
    
    return [rc[0] for rc in prof_counts], [rc[1] for rc in prof_counts]
